In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os

os.chdir("/content/drive/My Drive/Colab/assignment 2")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
myDataFrame2 = pd.read_csv('children.csv')
myDataFrame2.head()

,AgreementId,Version,Name,Region,Country,Peace Process,Peace Process Name,Stage,Signed Date,Agreement/Conflict Level,Conflict Nature,Agreement Status,Agreement Text
0,1923,2,Agreement between the two campaign teams regar...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2014-09-21,Intra,Government,Multiparty signed/agreed,Agreement between the Two Campaign Teams Regar...
1,864,1,Tokyo Declaration Partnership for Self-Relianc...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2012-07-08,InterIntra,Government,Multiparty signed/agreed,The Tokyo Declaration Partnership for Self-Rel...
2,848,1,Conclusions of the Conference on Afghanistan a...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Pre,2011-12-05,InterIntra,Government,Multiparty signed/agreed,The International Afghanistan Conference in Bo...
3,709,1,Renewed Commitment by the Afghan Government to...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2010-07-22,InterIntra,Government,Multiparty signed/agreed,A Renewed Commitment by the Afghan Government ...
4,708,1,The Resolution Adopted at the Conclusion of th...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Pre,2010-06-06,Intra,Government,Multiparty signed/agreed,Resolution Adopted at the Conclusion of the Na...


### Agreement Text
*   The last one to describe is the **Agreement Text** within this data. There are many stopwords need to be removed, and keywords need to be extracted.

In [ ]:
myDataFrame5=myDataFrame2[['Agreement Text']]
myDataFrame5

,Agreement Text
0,Agreement between the Two Campaign Teams Regar...
1,The Tokyo Declaration Partnership for Self-Rel...
2,The International Afghanistan Conference in Bo...
3,A Renewed Commitment by the Afghan Government ...
4,Resolution Adopted at the Conclusion of the Na...
...,...
313,The Peace and National Partnership Agreement 2...
314,National Dialogue Conference Outcomes Document...
315,"In the name of God, the Merciful, the Compassi..."
316,\r\n\r\nPreamble \r\n\r\nWe the people of Zim...


Create a new file for future use.

In [ ]:
myDataFrame5.to_csv('agreement.csv')

Delete all blank lines in the file and save as another file.

In [ ]:
f=open('agreement.csv')
g=open('re_agreement.csv','w')
try:
    while True:
        line=f.readline()
        if len(line)==0:
            break
        if line.count('\n')==len(line):
            continue
        g.write(line)
finally:
    f.close()
    g.close()

Clean the text with spacy method.

In [ ]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    ldatokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            ldatokens.append('URL') 
        elif token.orth_.startswith('@'):
            ldatokens.append('SCREEN_NAME')
        else:
            ldatokens.append(token.lower_)
    return ldatokens

Lemmatization with wordnet method.

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def lemma1(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
from nltk.stem.wordnet import WordNetLemmatizer
def lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Use nltk to remove stopwords.

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
  def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens]
    return tokens

# lda

This is the data prepared for lda model, which can also first show some high frenquency words in the text.

In [ ]:
import random
text_data = []
with open('re_agreement.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['party', 'express', 'appreciation', 'play', 'international', 'community', 'facilitate', 'political', 'technical', 'agreement', 'welcome', 'assurance', 'party', 'receive', 'support', 'implementation', 'agreement', 'engagement', 'government', 'national', 'unity']
['today', 'solemnly', 'declare', 'strategic', 'consensus', 'deepening', 'broadening', 'partnership', 'afghanistan', 'international', 'community', 'found', 'petersberg', 'years', 'building', 'share', 'achievement', 'years', 'recognise', 'security', 'well‐being', 'afghanistan', 'continue', 'affect', 'security', 'entire', 'region', 'beyond', 'afghanistan', 'international', 'community', 'strongly', 'commit', 'renew', 'partnership', 'transformation', 'decade']
[]
[]
['afghan', 'government', 'support', 'international', 'community', 'pursue', 'rates', 'sustainable', 'economic', 'growth', 'reducing', 'hunger', 'poverty', 'unemployment', 'promote', 'potential', 'private', 'sector', 'alongside', 'public', 'profit', 'sector', 'narcotic', 

Use for building matrix.

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

5 topics for lda model.

In [ ]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.016*"national" + 0.015*"article" + 0.012*"right" + 0.011*"shall"')
(1, '0.018*"shall" + 0.013*"government" + 0.011*"member" + 0.010*"president"')
(2, '0.026*"shall" + 0.013*"council" + 0.013*"government" + 0.010*"minister"')
(3, '0.022*"party" + 0.014*"force" + 0.013*"shall" + 0.010*"political"')
(4, '0.026*"shall" + 0.011*"right" + 0.009*"state" + 0.008*"ensure"')


3 topics for lda model.

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.012*"article" + 0.012*"force" + 0.011*"government" + 0.009*"arm"')
(1, '0.019*"shall" + 0.012*"right" + 0.011*"party" + 0.010*"government"')
(2, '0.035*"shall" + 0.011*"agreement" + 0.008*"commission" + 0.007*"sudan"')


10 topics for lda model.

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.025*"shall" + 0.017*"national" + 0.014*"sudan" + 0.011*"government"')
(1, '0.014*"national" + 0.014*"party" + 0.013*"shall" + 0.012*"political"')
(2, '0.036*"shall" + 0.017*"development" + 0.013*"right" + 0.011*"state"')
(3, '0.011*"determine" + 0.010*"right" + 0.010*"number" + 0.008*"colonel"')
(4, '0.022*"shall" + 0.017*"president" + 0.013*"government" + 0.009*"parliament"')
(5, '0.022*"right" + 0.015*"shall" + 0.012*"force" + 0.011*"arm"')
(6, '0.031*"shall" + 0.026*"article" + 0.023*"state" + 0.008*"party"')
(7, '0.017*"agreement" + 0.012*"government" + 0.007*"republic" + 0.006*"party"')
(8, '0.023*"shall" + 0.019*"agreement" + 0.012*"government" + 0.010*"bougainville"')
(9, '0.013*"implementation" + 0.011*"national" + 0.010*"order" + 0.010*"party"')


## Explore

### Use pyldavis to visualize different frequent words under different topics.

In [ ]:
!pip install pyldavis

儿童 5 主题

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

儿童 3 主题

In [ ]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


儿童 10 主题

In [ ]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
